In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_nfl = pd.read_csv('../input/nfl-elos/nfl_elo.csv')
df_curr_nfl = pd.read_csv('../input/nfl-elos/nfl_elo_latest.csv')

In [ ]:
df_nfl.head()

In [ ]:
df_curr_nfl.head()

In [ ]:
print('2020 Only: ' + str(len(df_curr_nfl)))
print('All Years: ' + str(len(df_nfl)))

Only going to take the years from 2000 onwards. Figure 20 years of data should be plenty while also discarding the older playstyle of the game.

In [ ]:
short_df = df_nfl[df_nfl.season >= 2000]

In [ ]:
short_df.head()

In [ ]:
func = lambda x: x - 2000
short_df.season = short_df.season.apply(func)

In [ ]:
short_df.head()

In [ ]:
X = short_df[['season', 'elo1_pre', 'elo2_pre', 'elo_prob1', 'elo_prob2', 'qb1_value_pre', 'qb2_value_pre']]
y = short_df[['score1', 'score2']]
# 

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from keras import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(50, input_dim=7, kernel_initializer='he_uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2))
model.compile(loss='mae', optimizer='adam')

In [ ]:
X_train.shape

In [ ]:
from keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

model.fit(X_train, y_train, validation_split=0.2, epochs=500, callbacks=[callback], verbose=1, batch_size=128)

In [ ]:
n = 525
k = n+1

row = X_test.iloc[n:k]
yhat = model.predict(row)
print(yhat[0])
print(y_test[n:k])

In [ ]:
arr = list()
iters = 1000
for i in range(iters):
    n = i
    k = n+1
    row = X_test.iloc[n:k]
    predict = model.predict(row)
    
    if (predict[0][0] > predict[0][1]):
        predict = 1
    else:
        predict = 2
        
    if (y_test[n:k].score1.iloc[0] > y_test[n:k].score2.iloc[0]):
        actual = 1
    else:
        actual = 2
        
    if predict == actual:
        arr.append(1)
    else:
        arr.append(0)
        
print('PREDICTION ACCURACY: ' + str((sum(arr)/iters)*100) + '%')